In [1]:
import cv2
import os
from ultralytics import YOLO
import torch
import numpy as np

In [2]:
image_path = "C:/Users/pragy/prototype/ml_models/assets/trial12.png" 
model = YOLO("C:/Users/pragy/prototype/ml_models/runs/detect/train2/weights/best.pt")
result = model(image_path)
annotated_image = result[0].plot()
cv2.imshow("pothole",annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\pragy\prototype\ml_models\assets\trial12.png: 256x416 7 potholess, 83.9ms
Speed: 16.6ms preprocess, 83.9ms inference, 321.6ms postprocess per image at shape (1, 3, 256, 416)


In [3]:
save_dir = r'C:\Users\pragy\prototype\ml_models\trial_assets'
os.makedirs(save_dir,exist_ok=True)
img = cv2.imread(image_path)

for i,box in enumerate(result[0].boxes.xyxy):
    x1,y1,x2,y2 = map(int,box[:4])
    cropped_img = img[y1+5:y2+5,x1+5:x2+5]

    crop_path = os.path.join(save_dir,f"cropped_pothole_{i}.jpg")
    cv2.imwrite(crop_path,cropped_img)
    print(f"Cropped image saved to {crop_path}")

Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_0.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_1.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_2.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_3.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_4.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_5.jpg
Cropped image saved to C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_6.jpg


In [4]:
image = cv2.imread(r"C:\Users\pragy\prototype\ml_models\trial_assets\cropped_pothole_1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("threshold",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
area_pixels = cv2.contourArea(largest_contour)

PIXEL_TO_M2 = 0.000045
area_m2 = area_pixels * PIXEL_TO_M2

print(f"Pothole area: {area_m2:.4f} m²")


Pothole area: 0.2308 m²


In [5]:
import math

perimeter = cv2.arcLength(largest_contour, True)
circularity = 4 * math.pi * area_pixels / (perimeter * perimeter)
print(f"Circularity: {circularity:.3f}")

if circularity > 0.85:
    shape = "Circle"
elif 0.5 < circularity <= 0.85:
    shape = "Oval"
else:
    shape = "Polygon or Irregular"

print(f"Pothole shape: {shape}")


Circularity: 0.683
Pothole shape: Oval
